In [ ]:
import pandas as pd
import re
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

query = 'SELECT * FROM public.overall_cleaned_base_and_pr_ef_policyef;'
data = pd.read_sql(query, con=engine)

In [ ]:
# Calculate total revenue (total premium paid per customer across all policies)
data['total_revenue'] = data['total premium payable']

# Calculate total revenue for each customer 
customer_total_revenue = data.groupby('customerid')['total_revenue'].sum()

# Calculate total number of purchases (policies) per customer
customer_total_purchases = data.groupby('customerid').size()

# Calculate Average Purchase Value (APV) for each customer
customer_apv = customer_total_revenue / customer_total_purchases

# Keep only one 'Churned' value per customer 
data_customer_level = data.drop_duplicates(subset='customerid', keep='first')

# Calculate customer-specific average purchase frequency (APF)
# APF is the total number of purchases divided by the number of unique customers
unique_customers = data_customer_level['customerid'].nunique()
customer_apf = customer_total_purchases / unique_customers

# Convert 'Churned' column to binary for churn rate calculation
data_customer_level['Churned_Binary'] = data_customer_level['Overall Churned'].apply(lambda x: 1 if x == 'Yes' else 0)

# Calculate churn rate (generalized for the cohort)
churned_customers = data_customer_level[data_customer_level['Churned_Binary'] == 1]['customerid'].nunique()
churn_rate = churned_customers / unique_customers

# Calculate average customer lifespan (ACL)
average_customer_lifespan = 1 / churn_rate if churn_rate != 0 else float('inf')

# Calculate customer-wise CLV with individualized APF
customer_clv = customer_apv * customer_apf * average_customer_lifespan

# Create a dataframe for customer-wise CLV and additional columns
customer_clv_df = pd.DataFrame({
    'customerid': customer_total_revenue.index,
    'Customer_APV': customer_apv.values,
    'Customer_APF': customer_apf.values,  
    'Churn_Rate': churn_rate,             
    'Average_Customer_Lifespan': average_customer_lifespan,  
    'CLV': customer_clv.values
})

print(customer_clv_df.head())

# Export the results to a PostgreSQL table
output_table_name = "customer_clv_overallbaseprappended"
customer_clv_df.to_sql(output_table_name, con=engine, if_exists='replace', index=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6020\1379280009.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_customer_level['Churned_Binary'] = data_customer_level['Overall Churned'].apply(lambda x: 1 if x == 'Yes' else 0)


  customerid  Customer_APV  Customer_APF  Churn_Rate  \
0    1000001       11975.0  6.700994e-07    0.454992   
1    1000002       43083.0  6.700994e-07    0.454992   
2    1000003       27060.0  6.700994e-07    0.454992   
3    1000004       58823.0  6.700994e-07    0.454992   
4    1000005        9971.0  6.700994e-07    0.454992   

   Average_Customer_Lifespan       CLV  
0                    2.19784  0.017636  
1                    2.19784  0.063451  
2                    2.19784  0.039853  
3                    2.19784  0.086633  
4                    2.19784  0.014685  


316

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}

connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Load only CustomerID and CLV from PostgreSQL
query = 'SELECT "customerid", "CLV" FROM "customer_clv_overallbaseprappended";'
clv_data = pd.read_sql(query, con=engine)

# Convert CustomerID to string (to avoid dtype mismatch)
clv_data['customerid'] = clv_data['customerid'].astype(str)

# Load the CSV file containing CustomerID and other data
csv_file_path = "RanCat_predictions_JFMAMJ(Final).csv"
csv_data = pd.read_csv(csv_file_path)

# Convert CustomerID to string (to ensure compatibility)
csv_data['customerid'] = csv_data['customerid'].astype(str)

# Merge only CLV data into the CSV file based on CustomerID
merged_data = csv_data.merge(clv_data, on='customerid', how="left")

# Save the merged data back to a new CSV file
output_csv_path = "RanCat_predictions_JFMAMJ(Final)_clv.csv"
merged_data.to_csv(output_csv_path, index=False)

print("Merged data saved to:", output_csv_path)

Merged data saved to: RanCat_predictions_JFMAMJ(Final)_clv.csv


In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Database connection setup
db_config = {
    'host': 'localhost',
    'database': 'postgres',
    'user': 'postgres',
    'password': 'kaviyam123',
    'port': '5432'
}

# Create database connection engine
connection_string = f"postgresql://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}"
engine = create_engine(connection_string)

# Load only CustomerID and CLV from PostgreSQL
query = 'SELECT "customerid", "CLV" FROM "customer_clv_overallbaseprappended";'
clv_data = pd.read_sql(query, con=engine)

# Convert CustomerID to string (to avoid dtype mismatch)
clv_data['customerid'] = clv_data['customerid'].astype(str)

# Load the CSV file containing CustomerID and other data
csv_file_path = "SOND_rancat45_55(Final).csv"
csv_data = pd.read_csv(csv_file_path)

# Convert CustomerID to string (to ensure compatibility)
csv_data['customerid'] = csv_data['customerid'].astype(str)

# Merge only CLV data into the CSV file based on CustomerID
merged_data = csv_data.merge(clv_data, on='customerid', how="left")

# Save the merged data back to a new CSV file
output_csv_path = "SOND_rancat45_55(Final)_clv.csv"
merged_data.to_csv(output_csv_path, index=False)

print("Merged data saved to:", output_csv_path)

Merged data saved to: SOND_rancat45_55(Final)_clv.csv
